<a href="https://colab.research.google.com/github/GihanMaheshWijesuriya/AngularPOSsytem/blob/productionBranch/smartyBotTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/sample_data/smartyLinkBotData.json').read()
intents = json.loads(data_file)
print

<function print>

In [21]:
for intent in intents['intents']:
    for pattern in intent['pattern']:

        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [22]:
classes = sorted(list(set(classes)))

print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

training = []

output_empty = [0] * len(classes)

55 documents
12 classes ['customer', 'family', 'goodbye', 'greeting', 'options', 'personal', 'profile', 'service_category_search', 'service_provider', 'service_provider_search', 'short', 'thanks']
83 unique lemmatized words ["'s", ',', 'a', 'about', 'age', 'allright', 'am', 'any', 'anyone', 'are', 'awesome', 'be', 'bos', 'brother', 'bye', 'ca', 'can', 'category', 'chatting', 'could', 'customer', 'day', 'do', 'eat', 'family', 'father', 'favorite', 'fill', 'food', 'for', 'from', 'going', 'good', 'goodbye', 'greeting', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'interest', 'is', 'it', 'know', 'later', 'location', 'me', 'mother', "n't", 'name', 'next', 'nice', 'number', 'offered', 'ok', 'provide', 'provider', 'see', 'service', 'subject', 'support', 't', 'tell', 'thank', 'thanks', 'thankz', 'that', 'the', 'there', 'time', 'to', 'want', 'what', 'where', 'who', 'why', 'you', 'your']


In [23]:
for doc in documents:

    bag = []

    pattern_words = doc[0]

    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [24]:
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)
print(hist)
print("model Created")

Training data created
Epoch 1/200
11/11 [==============================] - 0s 2ms/step - loss: 2.4157 - accuracy: 0.1660
Epoch 2/200
11/11 [==============================] - 0s 2ms/step - loss: 2.2473 - accuracy: 0.2886
Epoch 3/200
11/11 [==============================] - 0s 2ms/step - loss: 2.2246 - accuracy: 0.2878
Epoch 4/200
11/11 [==============================] - 0s 2ms/step - loss: 1.9966 - accuracy: 0.2652
Epoch 5/200
11/11 [==============================] - 0s 2ms/step - loss: 1.9706 - accuracy: 0.3138
Epoch 6/200
11/11 [==============================] - 0s 2ms/step - loss: 1.7078 - accuracy: 0.4415
Epoch 7/200
11/11 [==============================] - 0s 2ms/step - loss: 2.0561 - accuracy: 0.3859
Epoch 8/200
11/11 [==============================] - 0s 2ms/step - loss: 1.6909 - accuracy: 0.4762
Epoch 9/200
11/11 [==============================] - 0s 3ms/step - loss: 1.4696 - accuracy: 0.5393
Epoch 10/200
11/11 [==============================] - 0s 3ms/step - loss: 1.2803 - accu